In [1]:
import os
import scipy.io
import numpy as np
import mne
from glob import glob
from mne.datasets import sample

In [2]:
downloads_folder = os.path.expanduser("~/Downloads")  # Get the path to the "Downloads" directory
mat_files_folder = os.path.join(downloads_folder, "mat")

In [3]:
def read_mat_eeg(file_path):
    try:
        # Load MATLAB file
        mat = scipy.io.loadmat(file_path)

        # Access the EEG data under the 'data' key
        eeg_data = mat['data']

        # Check the shape of the data
        if len(eeg_data.shape) == 3:
            # Reshape data assuming it is in the format (trials, channels, samples)
            n_trials, n_channels, n_samples = eeg_data.shape

            # Concatenate trials to have a continuous data stream
            eeg_data = np.concatenate(eeg_data, axis=1)  # This concatenates along the second axis (samples)
            eeg_data = eeg_data.reshape(n_channels, n_trials * n_samples)
        else:
            raise ValueError(f"Unexpected data dimensions {eeg_data.shape}")

        # Define channel names and types
        ch_names = ['EEG ' + str(i) for i in range(n_channels)]
        ch_types = ['eeg'] * n_channels

        # Define the sampling frequency
        sfreq = 500  # Replace with the actual sampling frequency if available

        # Create MNE info structure
        info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

        # Create RawArray
        raw = mne.io.RawArray(eeg_data, info=info)
        
        # Extract labels if 'labels' key exists
        if 'labels' in mat:
            labels = mat['labels'].flatten()  # Ensure it is a 1D array
        else:
            raise ValueError("The key 'labels' was not found in the .mat file.")

        return raw, sfreq, labels

    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")
        return None, None, None

In [4]:


def visualize_3d_brain_model(raw):
    
    # Define the correct mapping from the current channel names to the desired channel names
    channel_mapping = {'EEG 0': 'Fp1','EEG 1': 'Fp2','EEG 2': 'F7','EEG 3': 'F3',
                     'EEG 4': 'Fz','EEG 5': 'F4', 'EEG 6': 'F8', 'EEG 7': 'FC5','EEG 8': 'FC1',
                    'EEG 9': 'FC2','EEG 10': 'FC6', 'EEG 11': 'T3',
                    'EEG 12': 'C3','EEG 13': 'Cz', 'EEG 14': 'C4', 'EEG 15': 'T4',
                    'EEG 16': 'CP5', 'EEG 17': 'CP1','EEG 18': 'CP2','EEG 19': 'CP6','EEG 20': 'T5',
                    'EEG 21': 'P3','EEG 22': 'Pz','EEG 23': 'P4','EEG 24': 'T6',
                    'EEG 25': 'PO3','EEG 26': 'PO4', 'EEG 27': 'O1', 'EEG 28': 'Oz',
                         'EEG 29': 'O2','EEG 30': 'A1',   'EEG 31': 'A2', }
    

   # Rename the channels in the raw object
    raw.rename_channels(channel_mapping)

    # Set the montage (standard 10-20 system) for EEG data after renaming channels
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)

    
     # Ensure 'fsaverage' is correctly installed and get its path
    subject = 'sample'
    fs_dir = mne.datasets.sample.data_path()
    subjects_dir = os.path.join(fs_dir, 'subjects')

    # Set the SUBJECTS_DIR environment variable
    os.environ['SUBJECTS_DIR'] = subjects_dir

    # Now check if the necessary files exist
    bem_dir = os.path.join(subjects_dir, subject, 'bem')
    head_surf_files = [
        'outer_skin.surf',
        'flash/outer_skin.surf',
        'fsaverage-head-sparse.fif',
        'fsaverage-head.fif'
    ]
    for file in head_surf_files:
        full_path = os.path.join(bem_dir, file)
        if not os.path.isfile(full_path):
            print(f'Missing file: {full_path}')

    mne.viz.set_3d_backend('pyvista')        
    
    # Plot the sensor locations, including the head surface
    fig = mne.viz.plot_alignment(
        raw.info,
        trans='fsaverage',
        subject=subject,
        subjects_dir=subjects_dir,
        dig=True,
        eeg=['original', 'projected'],
        show_axes=True,
        surfaces='head-dense',  # Use a denser head surface for better visualization (can be changed to 'head' for sparser)
       
    )

     # Set the 3D view of the figure
    mne.viz.set_3d_view(figure=fig, azimuth=90, elevation=90, distance=0.6)

# Extract the 3D positions of the EEG sensors and their corresponding names
    ch_pos = {ch_name: raw.info['chs'][idx]['loc'][:3] for idx, ch_name in enumerate(raw.info['ch_names'])}

    # Get the PyVista plotter from the figure
    plotter = fig.plotter

    # Offset for text annotation
    offset = np.array([0.0, 0.02, 0.0])  # Adjust this offset as needed

    label_actors = {}

    # Loop through each channel position and add a text label to the plotter
    for ch_name, pos in ch_pos.items():
        # Apply an offset to each position for better visibility
        text_pos = pos + offset
        # Here, pos is a NumPy array with 3 elements: x, y, and z
        plotter.add_point_labels([text_pos], [ch_name], point_size=20, font_size=20, text_color='black')

    # Render the plotter to show the text annotations
    plotter.render()

    return fig


In [5]:
file_paths = glob(os.path.join(mat_files_folder, '*.*'))

In [6]:
for file_path in file_paths: 
  raw_data, sfreq, labels = read_mat_eeg(file_path)  # Handling .mat file


# Now visualize the data
fig = visualize_3d_brain_model(raw_data)  

Creating RawArray with float64 data, n_channels=32, n_times=100000
    Range : 0 ... 99999 =      0.000 ...   199.998 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=100000
    Range : 0 ... 99999 =      0.000 ...   199.998 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=99000
    Range : 0 ... 98999 =      0.000 ...   197.998 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=99000
    Range : 0 ... 98999 =      0.000 ...   197.998 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=94000
    Range : 0 ... 93999 =      0.000 ...   187.998 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=92000
    Range : 0 ... 91999 =      0.000 ...   183.998 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=95000
    Range : 0 ... 94999 =      0.000 ...   189.998 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=100000
    Range : 0 ... 99999 =   